## Generalized Policy Iteration 
We write two algorithms as functions to be used with generic MDPs. The first of these is Policy iteration, and the second is Value iteration. Both of these methods are instances of Generalized Policy Iteration algorithms, where we evaluate a policy, then make the policy greedy with respect to the value function. 

## Policy Iteration: 
Here, we perform policy evaluation and improvement one after the other till we get a policy that is stable

In [1]:
def policy_iteration(states, action_space, reward_per_state, gamma, transition_probabilities): 
    theta = 1e-4
    V = [0 for i in range(len(states))] #initialize Value function
    pi = [None for i in range(len(states))] 
    max_iterations = 1e5
    for i in range(max_iterations): 
        optimal_policy = True 
        for j in range(max_iterations): 
            max_diff = 0 
            V_new = [0 for i in range(len(states))]
            for state in states: 
                val = reward_per_state[state] 
                #calculate discounted return 
                for next_state in states: 
                    val+=reward_per_state[next_state]* transition_probabilities[state][next_state][pi[state]] * (gamma * V[next_state])
                max_diff = max(max_diff, abs(V[s]-val))
                V[s] = val
            if max_diff < theta: 
                break 
        
        for state in states: 
            current_value = V[s]
            for action in action_space: 
                val = reward_per_state[state]
                for next_state in states: 
                    val+=reward_per_state[next_state]* transition_probabilities[state][next_state][action] * (gamma * V[next_state])
                if val > current_value: 
                    pi[state] = action 
                    V[state] = val 
                    optimal_policy = False
        if optimal_policy: 
            break 
    return V, pi 

### Value Iteration
This method is more efficient (in most cases) when compared to the prior because we perform a singular backup during the evaluation phase. Here, we simply pick the action that maximizes our value function with one backup step. This can be computationally cheaper to perform than the algorithm above. 
Please Note, in this case we only build at deterministic policies. More often than not, we're going to want to build stochastic policies instead. 

In [1]:
def value_iteration(action_space, states, reward_per_state, gamma, transition_probabilities): 
    max_iterations = 1e5 
    theta = 1e-4
    V = [0 for i in range(len(states))] #initialize Value function
    pi = [None for i in range(len(states))] #Initialize policy 
    
    #run until change in value is smaller than theta or max iterations counted 
    for i in range(max_iterations): 
        max_diff = 0 
        V_new = [0 for i in range(len(states))]
        for state in states: 
            max_val = 0 
            for action in action_space: 
                val = reward_per_state[state] 
                #calculate discounted return 
                for next_state in states: 
                    val+=reward_per_state[next_state]* transition_probabilities[state][next_state][action] * (gamma * V[next_state])
                if val > max_val: 
                    max_val = val
                #pick action that maximizes value 
                if V[state] < val: 
                    pi[state] = action_space[action]
            max_diff = max(max_diff, abs(V[state]-V_new[state]))
        V = V_new 
        if max_diff < theta: 
            break 
    
    return V, pi 

### Evaluating both GPI methods over different MDPs 
Over the next section we'll build some basic Markov Decision Processes and apply these two iterative methods to them. We can then compare runtime and convergence of the two. 